In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
!pip install pyarrow

In [3]:
df = pd.read_parquet("../data/yellow_tripdata_2024-01.parquet")

- Read the data for January. How many columns are there?



In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [5]:
len(df.columns)

19

- There are 19 columns.

In [6]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

# Q2. Computing duration



- Now let's compute the duration variable. It should contain the duration of a ride in minutes.

- What's the standard deviation of the trips duration in January?



In [7]:
df["duration"] = df["tpep_dropoff_datetime"]-df["tpep_pickup_datetime"]
np.std(df["duration"])

Timedelta('0 days 00:34:51.062862')

- Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

- What fraction of the records left after you dropped the outliers?

In [8]:
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


In [9]:
df["duration"]

0          19.800000
1           6.600000
2          17.916667
3           8.300000
4           6.100000
             ...    
2964619     8.616667
2964620    14.750000
2964621    19.000000
2964622    17.850000
2964623    15.083333
Name: duration, Length: 2964624, dtype: float64

In [10]:
df_new = df[(df.duration >= 1) & (df.duration <= 60)]


In [11]:
len(df_new)/len(df)

0.9778326020432945

- What fraction of the records left after you dropped the outliers?

- 90%

- 92%

- 95%

- **98%**

# Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.



- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)


In [12]:
columns_with_id = [col for col in df_new.columns if 'ID' in col]



df_new[columns_with_id] = df_new[columns_with_id].astype(str)



/tmp/ipykernel_21262/4134766128.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new[columns_with_id] = df_new[columns_with_id].astype(str)


In [14]:
features = pd.get_dummies(df_new[columns_with_id])

: 

In [26]:
df_list = df_new.to_dict(orient='records')

: 

Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?